# Kuramoto-Sakaguchi on the star graph

### Numerical integration of the dynamics on graph

In [15]:
from scipy.integrate import ode
import numpy as np

def integrate_dynamics(t0, t1, dt, dynamics, adjacency_matrix,
                       integrator, init_cond, *args, print_process_time=False):
    """

    :param t0:
    :param t1:
    :param dt:
    :param dynamics:
    :param adjacency_matrix:
    :param integrator:
    :param init_cond:
    :param args:
    :param print_process_time:
    :return:
    """
    r = ode(dynamics).set_integrator(integrator)
    r.set_initial_value(init_cond, t0).set_f_params(adjacency_matrix, *args)
    t = [t0]
    sol = [init_cond]
    time = timer.clock()

    for r.t in range(int(t1/dt)):
        if r.successful():
            # print(r.t+dt, r.integrate(r.t+dt))
            t.append(r.t + dt)
            sol.append(r.integrate(r.t + dt))
        # else:
        #     print("Integration was not successful for this step.")

    if print_process_time:
        print("Integration done. Time to process:",
              np.round((timer.clock()-time)/60, 5),
              "minutes", "(", np.round(timer.clock()-time, 5), " seconds)")
    return np.array(sol)

### Define Kuramoto-Sakaguchi dynamics and its dimension-reduction

In [16]:
def kuramoto_sakaguchi(t, theta, A, omega, sigma, alpha):
    """

    :param t:
    :param theta:
    :param omega:
    :param sigma:
    :param A:
    :param alpha:
    :param N:
    :return:
    """
    N = len(A[:, 0])
    return omega \
        + sigma/N*(np.cos(theta+alpha)*np.dot(A, np.sin(theta))
                   - np.sin(theta+alpha)*np.dot(A, np.cos(theta)))

def reduced_kuramoto_sakaguchi_star_2D(t, W, MAMp, N,
                                       omega1, omega2, coupling, alpha):
    """

        :param W: vector containing the variable of the problem
                  (Rp, Phi)
        :param t: time list
        :param N: Number of nodes
        :param omega1: natural frequency of the core
        :param omega2: natural frequency of the periphery
        :param coupling: sigma/N
        :return:
    """
    Rp = W[0]
    Phi = W[1]

    Nf = N - 1

    # Synchro of the periphery
    dRpdt = coupling*(1 - Rp**2)/2*np.cos(Phi - alpha)

    # Phase difference between the phase of the core and the mesoscopic phase
    # observable of the periphery
    dPhidt = omega1 - omega2 - coupling*(N - 1)*Rp*np.sin(Phi + alpha) \
        - coupling*(1+Rp**2)/(2*Rp)*np.sin(Phi - alpha)

    return np.array([dRpdt, dPhidt])

In [17]:
# Analytical results (see article)

def R_glob(Rp, Phi, N):
    return np.sqrt(1 + (N - 1) ** 2 * Rp ** 2 + 2 * (N - 1) * Rp * np.cos(Phi)) / N

def Phi_fixed_point_1(omega1, omega2, sigma, N, alpha):
    return np.arcsin((omega1 - omega2) /(sigma * np.sqrt(N ** 2 - 4 * (N - 1) * np.sin(alpha) ** 2))) \
               - np.arcsin(((N - 2) * np.sin(alpha)) / (np.sqrt(N ** 2 - 4 * (N - 1) * np.sin(alpha) ** 2)))

### Get transition

In [18]:
def get_data_kuramoto_sakaguchi_one_star(sigma_array, N, alpha,
                                         t0, t1, dt, t0_red,
                                         t1_red, dt_red, averaging,
                                         plot_temporal_series=0):
    """
    Generate data for different reduced Kuramoto-Sakaguchi dynamics
    on a star graph

    IMPORTANT: The initial conditions of the complete and the reduced dynamics
    are different to simplify the convergence to equilibrium.

    See Gomez-Gardenes 2011 to know how to integrate the system and get the
    complete hysteresis.

    :param sigma_array: Array of the coupling constant sigma
    :param N: Number of nodes
    :param alpha: Phase-lag between the oscillators' phases
    :param t0: Initial time
    :param t1: Final time
    :param dt: Time step
    :param t0_red: Initial time (reduced dynamics)
    :param t1_red: Final time (reduced dynamics)
    :param dt_red: Time step (reduced dynamics)
    :param averaging: number between 5 and 10. ex: 8
    :param plot_temporal_series: (boolean) Show temporal series (1) or not (0)
                                 of r, R, r1, R1, r2, R2 when the weights are
                                 uniform in M
    :return: R_dictionary
            R_dictionary is a dictionary of the form
                               Keys             Values
                             { "r",             [--- r ---],
                               "r1",            [--- r1---],
                               "r2",            [--- r2---],
                               "R",             [--- R ---],
                               "R1",            [--- R1---],
                               "R2",            [--- R2---],
                               sigma_array,    sigma_array,
                               omega_array,    omega_array
                               alpha,              alpha, }
            where the values [---X---] is an array of shape
            1 times len(sigma_list) of the order parameter X.
            R is the spectral observable (obtained with M_A: Z = M_A z)
    """

    R_dictionary = {}
    r_matrix = np.zeros((len(sigma_array), 2))
    r2_matrix = np.zeros((len(sigma_array), 2))

    R_matrix = np.zeros((len(sigma_array), 2))
    R2_matrix = np.zeros((len(sigma_array), 2))

    # 2 stands for the number of stable branches in the hysteresis
    # the first branch (j = 0) is the forward (bottom) branch
    # the second branch (j = 1) is the backward (top) branch

    n1, n2 = 1, N-1

    A = nx.to_numpy_array(nx.star_graph(n2))
    K = np.diag(np.sum(A, axis=0))
    M = np.array([np.concatenate([[1], np.zeros(n2)]),
                  np.concatenate([[0], np.ones(n2) / n2])])

    omega1 = np.diag(K)[0]
    omega2 = np.diag(K)[1]
    omega_array = np.array([omega1, omega2])
    omega = np.array([omega1] + n2 * [omega2])
    Omega = (omega1 + (N-1)*omega2)/N
    omega_CM = np.array([omega1-Omega] + n2 * [omega2-Omega])

    print(sigma_critical(omega1, omega2, N, alpha))


    """ Get the top branch (backward branch) """
    j = 1

    theta0 = np.linspace(0, 2*np.pi*(1-1/N), N) # np.random.normal(0, 0.001, N)
    Zp = np.mean(np.exp(1j*theta0[1:]))
    Rp = np.absolute(Zp)
    Phip = np.angle(Zp)
    W0 = [Rp, Phip]

    print(W0)
    #  np.linspace(0, 3, N)  #
    # print(np.absolute(np.mean(np.exp(1j*theta0))))

    for i in tqdm(range(len(sigma_array))):
        time.sleep(1)

        sigma = sigma_array[-1 - i]

        # Integrate complete dynamics

        args_kuramoto = (omega_CM, N*sigma, alpha)
        # *N, to cancel the N in the definition of the dynamics "kuramoto"
        # in my code.
        kuramoto_sol = integrate_dynamics(t0, t1, dt, kuramoto_sakaguchi, A,
                                          "vode", theta0,
                                          *args_kuramoto)

        r2 = np.absolute(
            np.sum(M[1, :] * np.exp(1j * kuramoto_sol),
                   axis=1))
        r = np.absolute(
            np.sum(
                (n1 * M[0, :] + n2 * M[1, :]) * np.exp(1j * kuramoto_sol),
                axis=1)) / N

        r2_mean = np.sum(r2[averaging * int(t1 // dt) // 10:]
                         ) / len(r2[averaging * int(t1 // dt) // 10:])
        r_mean = np.sum(r[averaging * int(t1 // dt) // 10:]
                        ) / len(r[averaging * int(t1 // dt) // 10:])

        # Integrate reduced dynamics

        MAMp = multi_dot([M, A, pinv(M)])
        args_red_kuramoto_sakaguchi = (N, omega1, omega2, sigma, alpha)
        red_kuramoto_sol = \
            integrate_dynamics(t0_red, t1_red, dt_red,
                               reduced_kuramoto_sakaguchi_star_2D,
                               MAMp, "vode", W0,
                               *args_red_kuramoto_sakaguchi)

        R2 = red_kuramoto_sol[:, 0]
        Phi = red_kuramoto_sol[:, 1]

        R = np.absolute(np.exp(1j * Phi) + n2 * R2) / N

        R2_mean = np.sum(R2[averaging * int(t1 // dt) // 10:]
                         ) / len(R2[averaging * int(t1 // dt) // 10:])
        Phi_mean = np.sum(Phi[averaging * int(t1 // dt) // 10:]
                          ) / len(Phi[averaging * int(t1 // dt) // 10:])
        R_mean = np.sum(R[averaging * int(t1 // dt) // 10:]
                        ) / len(R[averaging * int(t1 // dt) // 10:])

        r_matrix[-1-i, j] = r_mean
        r2_matrix[-1-i, j] = r2_mean

        R_matrix[-1-i, j] = R_mean
        R2_matrix[-1-i, j] = R2_mean

        # W0 = [R_mean, Phi_mean]
        theta0 = kuramoto_sol[-1, :]
        Zp = np.mean(np.exp(1j * theta0[1:]))
        Rp = np.absolute(Zp)
        Phip = np.angle(Zp)
        W0 = [Rp, Phip]

        print(len(r), len(R))

        if plot_temporal_series:
            import matplotlib.pyplot as plt

            plt.figure(figsize=(8, 8))

            second_community_color = "#f16913"
            reduced_second_community_color = "#fdd0a2"
            ylim = [-0.02, 1.1]

            plt.subplot(211)
            plt.suptitle("$\\sigma = {}, \\omega_1 = {}, \\omega_2 = {}$"
                         .format(np.round(sigma_array[-1 - i], 3),
                                 np.round(omega1, 3),
                                 np.round(omega2, 3)), y=1.0)
            plt.plot(r, color="k", label="Complete spectral")
            plt.plot(R, color="grey", label="Reduced spectral")
            plt.plot(r_mean * np.ones(int(t1 // dt)), color="r")
            plt.plot(R_mean * np.ones(int(t1 // dt)), color="orange")
            plt.ylim(ylim)
            plt.ylabel("$R$", fontsize=12)
            plt.legend(loc=1, fontsize=10)

            plt.subplot(212)
            plt.plot(r2, color=second_community_color)
            plt.plot(R2, color=reduced_second_community_color)
            plt.ylabel("$R_2$", fontsize=12)
            plt.xlabel("$t$", fontsize=12)
            plt.ylim(ylim)

            plt.tight_layout()

            plt.show()

    R_dictionary["r"] = r_matrix.tolist()
    R_dictionary["r2"] = r2_matrix.tolist()

    R_dictionary["R"] = R_matrix.tolist()
    R_dictionary["R2"] = R2_matrix.tolist()

    R_dictionary["sigma_array"] = sigma_array.tolist()
    R_dictionary["omega_array"] = omega_array.tolist()
    R_dictionary["alpha"] = alpha

    return R_dictionary


### Simulation

In [19]:
# Predictions for the Kuramoto-Sakaguchi dynamics on a star graph when the core has one
# natural frequency and the periphery nodes have all the same natural frequency
from numpy.linalg import pinv
from tqdm import tqdm
import json
import tkinter.simpledialog
from tkinter import messagebox
import time as timer
import networkx as nx

# Time parameters
t0, t1, dt = 0, 3000, 1
t0_red, t1_red, dt_red = 0, 3000, 1


# Structural parameter of the star
q = 2
Nf = 10
sizes = [1, Nf]
N = sum(sizes)
n1 = sizes[0]
n2 = sizes[1]
averaging = 9

sigma_array = np.linspace(0.75, 0.95, 20)

alpha = 0.5

# Important
# we make N times sigma in
# dynamics/get_reduction_errors -> get_data_kuramoto_one_star !!!
# This is only to remove the division by N
# in the definition of the complete and reduced dynamics
R_dictionary = get_data_kuramoto_sakaguchi_one_star(sigma_array, N, alpha,
                                                    t0, t1, dt, t0_red,
                                                    t1_red, dt_red,
                                                    averaging,
                                                    plot_temporal_series=0)

line1 = "t0 = {}\n".format(t0)
line2 = "t1 = {}\n".format(t1)
line3 = "deltat = {}\n".format(dt)
line4 = "Number of nodes : N = {}\n".format(N)
line5 = "Sizes : [n1, n2] = {}\n".format(sizes)
line6 = "adjacency_matrix_type = star graph\n"
line7 = "theta0 and W0 = see function get_data_kuramoto_sakaguchi_one_star\n"
line8 = "sigma_array = np.array({}, {}, {})\n".format(np.round(sigma_array[0],
                                                               3),
                                                      np.round(sigma_array[-1],
                                                               3),
                                                      len(sigma_array))
line9 = "alpha = {}\n".format(alpha)
line10 = "R_dictionary is a dictionary of the form \n\n " \
         "Keys (str)      Values         \n " \
         "{ r,             [[--- r ---]], \n" \
         "  r1,            [[--- r1---]], \n" \
         "  r2,            [[--- r2---]], \n" \
         "  R,             [[--- R ---]], \n" \
         "  ...                ... \n" \
         "  sigma_array,    sigma_array," \
         "  omega_array,    omega_array = [omega1(core), omega2(periph)] \n" \
         "  alpha,          alpha \n" \
         "  where the values [[---X---]] is an array of shape \n" \
         "  nb_initial_condition times len(sigma_array)of the order" \
         " parameter X\n " \
         "  R is the spectral observable (obtained with M_A: Z = M_A z)"

timestr = time.strftime("%Y_%m_%d_%Hh%Mmin%Ssec")

if messagebox.askyesno("Python",
                       "Would you like to save the parameters, "
                       "the data and the plot?"):
    window = tkinter.Tk()
    window.withdraw()  # hides the window
    file = tkinter.simpledialog.askstring("File: ", "Enter your file name")

    f = open('data/kuramoto_sakaguchi/{}_{}_dataset_informations'
             '_kuramoto_sakaguchi_star_2D.txt'.format(timestr, file),
             'w')
    f.writelines(
        [line1, line2, line3, "\n", line4, line5, line6, line7, line8,
         line9, "\n", line10])

    f.close()

    with open('data/kuramoto_sakaguchi/'
              '{}_{}_data_R_dictionary_kuramoto_sakaguchi_star_2D'
              '.json'.format(timestr, file),
              'w') as outfile:
        json.dump(R_dictionary, outfile)




  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

  1%|▊                                                                                 | 1/100 [00:32<53:33, 32.46s/it]

  2%|█▋                                                                                | 2/100 [01:04<52:45, 32.31s/it]

  3%|██▍                                                                               | 3/100 [01:35<51:35, 31.91s/it]

  4%|███▎                                                                              | 4/100 [02:08<51:43, 32.32s/it]

  5%|████                                                                              | 5/100 [02:42<51:45, 32.69s/it]

  6%|████▉                                                                             | 6/100 [03:16<52:02, 33.21s/it]

  7%|█████▋                                                                            | 7/100 [03:49<51:18, 33.10s/it]

  8%|██████▌                  

 67%|██████████████████████████████████████████████████████▎                          | 67/100 [28:07<08:15, 15.01s/it]

 68%|███████████████████████████████████████████████████████                          | 68/100 [28:18<07:26, 13.96s/it]

 69%|███████████████████████████████████████████████████████▉                         | 69/100 [28:31<07:02, 13.61s/it]

 70%|████████████████████████████████████████████████████████▋                        | 70/100 [28:44<06:37, 13.25s/it]

 71%|█████████████████████████████████████████████████████████▌                       | 71/100 [28:56<06:18, 13.07s/it]

 72%|██████████████████████████████████████████████████████████▎                      | 72/100 [29:06<05:37, 12.06s/it]

 73%|███████████████████████████████████████████████████████████▏                     | 73/100 [29:18<05:22, 11.94s/it]

 74%|███████████████████████████████████████████████████████████▉                     | 74/100 [29:29<05:06, 11.77s/it]

 75%|███████████████████████████

FileNotFoundError: [Errno 2] No such file or directory: 'data/kuramoto_sakaguchi/2020_02_01_04h29min38sec_test_dataset_informations_kuramoto_sakaguchi_star_2D.txt'

In [20]:
import matplotlib.pyplot as plt
import numpy as np


def left_bottom_axis_settings(ax, xlabel, ylabel, xlim, ylim, x_coord, y_coord,
                              fontsize, labelsize, linewidth):
    plt.xlabel(xlabel, fontsize=fontsize)
    ylab = plt.ylabel(ylabel, fontsize=fontsize)
    ylab.set_rotation(0)
    ax.xaxis.set_label_coords(*x_coord)
    ax.yaxis.set_label_coords(*y_coord)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.tick_params(axis='both', which='major', labelsize=labelsize)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    for axis in ['bottom', 'left']:
        ax.spines[axis].set_linewidth(linewidth-1)


def plot_transitions_complete_vs_reduced(x_array, r_matrix, R_matrix,
                                         complete_color, reduced_color,
                                         alpha, marker, s, linewidth,
                                         nb_instances):

    # mean_r = np.mean(r_matrix, axis=0)
    mean_R = np.mean(R_matrix, axis=0)
    # std_r = np.std(r_matrix, axis=0)
    std_R = np.std(R_matrix, axis=0)

    for i in range(0, nb_instances):
        if i == 0:
            plt.scatter(x_array, r_matrix[i], color=complete_color, s=s,
                        alpha=0.9, marker=marker)
        else:
            plt.scatter(x_array, r_matrix[i], color=complete_color, s=s,
                        alpha=0.9, marker=marker)

    plt.fill_between(x_array, mean_R - std_R, mean_R + std_R,
                     color=reduced_color, alpha=alpha)
    plt.plot(x_array, mean_R, color=reduced_color, linewidth=linewidth,
             linestyle='-')


def plot_multiple_transitions_complete_vs_reduced(ax, sigma_array, r_array,
                                                  R_array, complete_color,
                                                  reduced_color, linewidth):
    ax.plot(sigma_array, r_array, color=complete_color,
            label="$\\langle R^{com} \\rangle_t$", linewidth=linewidth)
    ax.plot(sigma_array, R_array,
            color=reduced_color, linestyle="-",
            label="$\\langle R^{red} \\rangle_t$", linewidth=linewidth)
    ax.fill_between(sigma_array, r_array, R_array,
                    color=complete_color, alpha=0.2)

In [21]:
first_community_color = "#2171b5"
second_community_color = "#f16913"
reduced_first_community_color = "#9ecae1"
reduced_second_community_color = "#fdd0a2"
reduced_third_community_color = "#a1d99b"
reduced_fourth_community_color = "#9e9ac8"
total_color = "#525252"
fontsize = 12
inset_fontsize = 9
fontsize_legend = 12
labelsize = 12
inset_labelsize = 9
linewidth = 2
s = 20
alpha = 0.5
marker = "."
x_lim_kb = [0.78, 2.52]
y_lim_kb = [0, 1.02]


theoretical = 0


# with open('data/kuramoto/2019_10_03_17h05min34sec_small_test_data
#           _R_dictionary_'
#           'kuramoto_star_2D.json'
#           ) as json_data:
#     R_dictionary = json.load(json_data)
# with open('data/kuramoto_sakaguchi/2019_12_10_18h09min57sec_1_data_R_dictionary_kuramoto_sakaguchi_star_2D.json'
#           ) as json_data:
#     R_dictionary = json.load(json_data)

# with open('data/kuramoto_sakaguchi/2019_12_10_18h15min53sec_2_data_R_dictionary_kuramoto_sakaguchi_star_2D.json'
#           ) as json_data:
#     R_dictionary = json.load(json_data)

# with open('data/kuramoto_sakaguchi/2019_12_10_18h18min02sec_3_data_R_dictionary_kuramoto_sakaguchi_star_2D.json'
#           ) as json_data:
#     R_dictionary = json.load(json_data)

# with open('data/kuramoto_sakaguchi/2019_12_10_18h19min52sec_4_data_R_dictionary_kuramoto_sakaguchi_star_2D.json'
#           ) as json_data:
#     R_dictionary = json.load(json_data)

# with open('data/kuramoto_sakaguchi/2019_12_10_18h20min14sec_5_data_R_dictionary_kuramoto_sakaguchi_star_2D.json'
#           ) as json_data:
#     R_dictionary = json.load(json_data)

# with open('data/kuramoto_sakaguchi/2019_12_10_18h20min59sec_6_data_R_dictionary_kuramoto_sakaguchi_star_2D.json'
#           ) as json_data:
#     R_dictionary = json.load(json_data)

# with open('data/kuramoto_sakaguchi/2019_12_10_18h21min31sec_7_data_R_dictionary_kuramoto_sakaguchi_star_2D.json'
#           ) as json_data:
#     R_dictionary = json.load(json_data)
# with open('data/kuramoto_sakaguchi/2019_12_10_18h21min44sec_8_data'
#           '_R_dictionary_kuramoto_sakaguchi_star_2D.json'
#           ) as json_data:
#     R_dictionary = json.load(json_data)
with open('data/kuramoto_sakaguchi/2019_12_11_12h55min31sec_alpha_0_9_data_R_dictionary_kuramoto_sakaguchi_star_2D.json'
          ) as json_data:
    R_dictionary = json.load(json_data)

sigma_array = np.array(R_dictionary["sigma_array"])
omega_array = np.array(R_dictionary["omega_array"])
alpha = np.array(R_dictionary["alpha"])

nb_sigma = len(sigma_array)
nb_init_cond = 2
# Because we choose different initial conditions for the two stable branch
# of the hysteresis

r_s_matrix = np.array(R_dictionary["r"]).T
R_s_matrix = np.array(R_dictionary["R"]).T


def R_glob(Rp, Phi, N):
    return np.sqrt(1 + (N-1)**2*Rp**2 + 2*(N-1)*Rp*np.cos(Phi))/N


def Phi_fixed_point_1(omega1, omega2, sigma, N, alpha):
    return np.arcsin((omega1 - omega2) /
                     (sigma * np.sqrt(N**2 - 4*(N-1)*np.sin(alpha)**2)))\
           - np.arcsin(((N-2)*np.sin(alpha)) /
                       (np.sqrt(N**2 - 4*(N-1)*np.sin(alpha)**2)))


Rp_fixed_point_1 = 1
sigma_array_fp1 = np.linspace(0.5, 2.5, 10000)


def R_top_branch(omega1, omega2, sigma, N, alpha):
    return R_glob(1, Phi_fixed_point_1(omega1, omega2, sigma, N, alpha), N)


def sigma_critical(omega1, omega2, N, alpha):
    return (omega1 - omega2)/np.sqrt(N**2 - 4*(N-1)*np.sin(alpha)**2)


if theoretical:
    plt.figure(figsize=(6, 3))

    plt.subplot(121)
    sigma_array_a = np.linspace(9/11+0.002, 2.5, 1000000)
    alpha_array = [-np.pi/2, -1, 0, 1, np.pi/2]
    ax = plt.gca()
    for alpha in alpha_array:
        plt.plot(sigma_array_a, R_top_branch(10, 1, sigma_array_a, 11, alpha),
                 linewidth=linewidth, linestyle="-",
                 label="$\\alpha = {}$".format(np.round(alpha, 2)))
        sigma_critical_value = sigma_critical(10, 1, 11, alpha)
        R_critical = R_top_branch(10, 1, sigma_critical_value, 11, alpha)
        plt.scatter(sigma_critical_value, R_critical, s=60)
    plt.legend(loc=4, fontsize=fontsize_legend)
    plt.xlabel("$\\sigma$", fontsize=fontsize)
    ylab = plt.ylabel("$R^*$", fontsize=fontsize, labelpad=10)
    plt.tick_params(axis='both', which='major', labelsize=labelsize)
    ylab.set_rotation(0)
    plt.ylim([0.8, 1.02])

    plt.subplot(122)
    # sigma_array_a = [9/11+0.002, 0.9, 1, 1.1, 2]
    sigma_array_a = [9/11+0.002, 1, 2]
    alpha_array = np.linspace(-np.pi/2, np.pi/2, 100000)
    ax = plt.gca()
    for sigma in sigma_array_a:
        plt.plot(alpha_array, R_top_branch(10, 1, sigma, 11, alpha_array),
                 linewidth=linewidth, linestyle="-",
                 label="$\\sigma = {}$".format(np.round(sigma, 2)))
    plt.legend(loc=4, fontsize=fontsize_legend)
    plt.xlabel("$\\alpha$", fontsize=fontsize)
    ylab = plt.ylabel("$R^*$", fontsize=fontsize, labelpad=10)
    plt.tick_params(axis='both', which='major', labelsize=labelsize)
    ylab.set_rotation(0)
    ax.xaxis.set_major_locator(plt.MultipleLocator(np.pi / 2))
    # ax.xaxis.set_major_formatter(plt.FuncFormatter(multiple_formatter()))
    plt.ylim([0.8, 1.02])

    plt.tight_layout()
    plt.show()

else:
    fig = plt.figure(figsize=(4, 4))

    # Kuramoto on star graph
    ax = plt.subplot(111)
    # ax.title.set_text('$(a)$')
    # ax.title.set_position((0.5, 1.05))
    colors_complete = ["#252525", reduced_third_community_color]
    colors_reduced = ["#969696", reduced_fourth_community_color]
    # for i in range(2):
    i = 1
    plt.scatter(sigma_array, r_s_matrix[i], color=colors_complete[i],
                marker=marker, s=s+80, linewidth=linewidth)

    plt.scatter(sigma_array, R_s_matrix[i], color=colors_reduced[i],
                marker=marker, s=s, linewidth=linewidth)

    plt.plot(sigma_array_fp1, R_top_branch(10, 1, sigma_array_fp1, 11, alpha),
             linewidth=linewidth-1, color="r", linestyle="-")
    left_bottom_axis_settings(ax, "$\\sigma$", "$R$", x_lim_kb, y_lim_kb,
                              (0.5, -0.15), (-0.2, 0.45), fontsize,
                              labelsize, linewidth)
    plt.xticks([0.5, 1, 1.5, 2, 2.5])

    plt.tight_layout()
    plt.show()
    # if messagebox.askyesno("Python",
    #                        "Would you like to save the plot ?"):
    #     window = tkinter.Tk()
    #     window.withdraw()  # hides the window
    #   file = tkinter.simpledialog.askstring("File: ", "Enter your file name")
    #     timestr = time.strftime("%Y_%m_%d_%Hh%Mmin%Ssec")
    #     # fig.savefig("data/kuramoto/"
    #     #             "{}_{}_complete_vs_reduced_kuramoto_star_2D"
    #     #             ".png".format(timestr, file))
    #     fig.savefig("data/kuramoto_sakaguchi/"
    #                 "{}_{}_complete_vs_reduced_kuramoto_star_2D"
    #                 ".pdf".format(timestr, file))

FileNotFoundError: [Errno 2] No such file or directory: 'data/kuramoto_sakaguchi/2019_12_11_12h55min31sec_alpha_0_9_data_R_dictionary_kuramoto_sakaguchi_star_2D.json'